We are working with the Sign Language MNIST dataset.
First we need to load the dataset for training and testing from the CSV files.

In [1]:
import numpy as np
import pandas as pd

training_df = pd.read_csv("C:\Machine Learning\CSCI4050U_FinalProject\dataset\sign_mnist_train\sign_mnist_train.csv") # Need to find a way to provide a relative path
testing_df = pd.read_csv("C:\Machine Learning\CSCI4050U_FinalProject\dataset\sign_mnist_test\sign_mnist_test.csv")

Extract Labels and Features

In [2]:
y_train = training_df['label']
training_df.drop(['label'], axis=1, inplace=True)

y_test = testing_df['label']
testing_df.drop(['label'], axis=1, inplace=True)

Define Constrants and reshape the data

In [3]:
# Adjust parameters as needed
size = 28
channels = 1
batch = 128
epochs = 100

X_train = training_df.values.reshape(training_df.shape[0], size, size, channels)
X_test = testing_df.values.reshape(testing_df.shape[0], size, size, channels)

Data Augmentation and Generators

In [4]:
# Link to instructions for getting tensorflow working
# https://www.tensorflow.org/install/pip#windows-native_1

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    width_shift_range=.2,
    height_shift_range=.2,
    rotation_range=15,
    brightness_range=[0.8, 1.2],
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

X_train = train_datagen.flow(X_train, y_train, batch_size=batch)
X_test = test_datagen.flow(X_test, y_test)

Base Model

In [10]:
from typing import Tuple
from lightning.pytorch import LightningModule
from torch import nn
import torch
from torch import Tensor
import torchmetrics

class BaseModel(LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.accuracy = torchmetrics.classification.Accuracy(
            task="multiclass",
            num_classes=num_classes)
        self.model = self.build_model()
        
    def build_model(self):
        raise Exception("Not yet implemented")

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def forward(self, x):
        return self.model(x)
    
    def loss(self, logits, target):
        return nn.functional.cross_entropy(logits, target)
    
    def shared_step(self, mode:str, batch:Tuple[Tensor, Tensor], batch_index:int):
        x, target = batch
        output = self.forward(x)
        loss = self.loss(output, target)
        self.accuracy(output, target)
        self.log(f"{mode}_step_acc", self.accuracy, prog_bar=True)
        self.log(f"{mode}_step_loss", loss, prog_bar=False)
        return loss
    
    def training_step(self, batch, batch_index):
        return self.shared_step('train', batch, batch_index)
    
    def validation_step(self, batch, batch_index):
        return self.shared_step('val', batch, batch_index)
    
    def test_step(self, batch, batch_index):
        return self.shared_step('test', batch, batch_index)

Helper Functions from Assignment 2

In [6]:
import torchinfo
def describe(model, **kwargs):
    return torchinfo.summary(model,
                             input_size=(batch_size, 1, 28, 28),
                             col_names=['input_size', 'output_size', 'kernel_size', 'num_params'],
                             row_settings=['ascii_only'])

In [7]:
import shutil
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer, seed_everything

def train(model):
    name = model.__class__.__name__
    shutil.rmtree(f'./lightning_logs/{name}', ignore_errors=True)
    seed_everything(0, workers=True)
    logger = CSVLogger('./lightning_logs', name=name)
    trainer = Trainer(max_epochs=3, logger=logger, deterministic=True)
    trainer.fit(model,
                train_dataloaders=training_dataloader,
                val_dataloaders=val_dataloader)

In [8]:
def show_metrics(name):
    df = pd.read_csv(f'./lightning_logs/{name}/version_0/metrics.csv')
    df.set_index('step', inplace=True)
    ax = df[['train_step_acc']].dropna().plot()
    df[['val_step_acc']].dropna().plot(ax=ax)
    return df[['val_step_acc']].dropna().round(2)

Convolution Net Model

In [11]:
class ConvNet(BaseModel):
    def __init__(self, num_classes, num_kernels, kernel_size, pool_size):
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.pool_size = pool_size
        super().__init__(num_classes)